In [2]:
import pandas as pd
import xgboost
import lightgbm
import catboost
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid, KFold, train_test_split
from sklearn.preprocessing import StandardScaler
import imblearn

<frozen importlib._bootstrap>:228: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


- 파이프라인 셋 / 업데이트
- 파라미터 셋 / 업데이트

In [2]:
class Test:
    def __init__(self):
        params_rf ={
            'n_estimators' : [100],
            'max_depth':[2,8,10,30],
            'min_samples_leaf':[2,8,12,20],
            'min_samples_split':[8,16,20]
        }
        params_xgb = {
            "max_depth": [10,30,50],
            "min_child_weight" : [1,3,6,10],
            "n_estimators": [200,300,500,1000],
            'learning_rate': [0.01]
        }
        params_lgb = {
            "learning_rate" : [0.01,0.1,0.2,0.3,0.4,0.5],
            "max_depth" : [25, 50, 75],
            "num_leaves" : [100,300,500,900,1200],
            "n_estimators" : [100, 200, 300,500,800,1000],
            "learning_rate" : [0.01,0.1,0.2,0.3,0.4,0.5]
        }
        params_cat = {
            "learning_rate" : '',
            "random_strength" : '',
            "L2_regulariser " : ''
        }



In [3]:
df = pd.read_csv('D:/kkbox-churn-prediction-challenge/data/churn_comp_refresh/df_target3.csv')

In [5]:
df.drop(columns=['Unnamed: 0'], inplace=True)

y = df['is_churn']
X = df.iloc[:,1:]

X = X.fillna(-1)
y = y.fillna(-1)

X = pd.get_dummies(X, columns=['city', 'gender', 'registered_via'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [6]:
df.shape

(955126, 9)

In [8]:
xgb = xgboost.XGBClassifier()

params_xgb = {
            "max_depth": [6,10,30,50],
            "min_child_weight" : [1,3,6,10],
            "n_estimators": [10,50,100,300],
            'learning_rate': [0.01]
        }

gs = GridSearchCV(estimator=xgb, param_grid=params_xgb, scoring='roc_auc', n_jobs=-1)

In [9]:
gs.fit(X_train, y_train)

In [ ]:
print('best parameters : ', gs.best_params_)
print('best score : ', gs.best_score_)
em = gs.best_estimator_
pred = em.predict(X_test)
accuracy_score(y_test, pred)